In [1]:
import pandas as pd
import urllib2
from xml.etree.ElementTree import parse

In [2]:
# open rss list
rss_list = pd.read_table('rss_list.txt', sep=',')

In [ ]:
# TODO add log files -> work, crashes
# TODO add saving to s3
# TODO get rss_list from file
# TODO saving to sql database (?)

In [21]:
def get_rss(url):
    # function get data from rss url -> return readed root
    # required urllib2
    try:
        tree = parse(urllib2.urlopen(url))
        root = tree.getroot()
        return root
    except:
        # change to save in crashes log file
        print "there is problem with parsing this url"

In [41]:
def rss_parse(root, site_name):
    # function parsed downloaded rss data -> return title, date, link, description, category
    # required xml.tree
    # TODO: parse enclosure url (?) for pic's
    
    title = []
    date = []
    link = []
    description = []
    category = []
    site = []
    
    for item in root.iterfind('channel/item'):
        title.append(item.findtext('title'))
        date.append(item.findtext('pubDate'))
        link.append(item.findtext('link'))
        description.append(item.findtext('description').strip())
        category.append(item.findtext('category'))
        site.append(site_name)
    
    return [title, date, link, description, category, site]

In [46]:
def save_parsed(parsed_list, path, filename, mode):
    # function get pandas dataframe from parsed data and save it to csv file -> return csv file
    # required pandas
    # using mode (?) 'w', 'a'???
    # saving to s3 in separate function (?)
    
    # dataframe from list of lists
    df = pd.DataFrame(parsed_list)
    # transpose dataframe for getting wright structure
    df = df.transpose()
    # append column names
    df.columns=['title', 'date', 'link', 'description', 'category', 'site']
    # save as csv file
    df.to_csv(path+filename, mode=mode, encoding='utf8', index=False)
    # clear memory from df
    del df

In [47]:
save_parsed(rss_parse(get_rss('https://lenta.ru/rss/news'),'lenta'), '','test_rss.csv', 'w')